___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

### QuantConnect Trading Basics Solutions

**Note, these solutions are meant to be run on the QuantConnect platform**

**Task**
**Invest \\$10,000 in Microsoft (MSFT) on Jan 1st, 2000 and run the simulation until Jan 1st 2020 with a buy and hold strategy (e.g. buy $10,000 of MSFT stock on Jan 1st 2000 and then just hold until Jan 1st 2020.**

In [ ]:
class BuyMicrosoft(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2000, 1, 1)  # We want to invest on the first January 2000
        self.SetEndDate(2020, 1, 1)  # End simulation on 1.1.2020
        self.SetCash(10000)  #  10,000$ are available
        
        # Let's say we want to invest in Microsoft. Thus we add Microsoft to our portfolio list.
        # As we want to buy and hold a daily resolution is enough
        self.AddEquity("MSFT", Resolution.Daily)  
        
    def OnData(self, data):
        ''' OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
            Arguments:
                data: Slice object keyed by symbol containing the stock data
        '''
      
        # Check if we're not invested and then invest everything in Microsoft.      
        if not self.Portfolio.Invested:
            self.SetHoldings("MSFT", 1)

**Task**  
* **Backtest Dates: Jan 1 2010 to Jan 1 2020**
* **Starting Cash: \\$10k**
* **Starting with 10k in USD Cash, Set 100% of your portfolio to MSFT. Using a Stop Market Order, sell 10 shares of MSFT if the price ever rises to 150% of your initial purchase price.**


In [ ]:
class BuyMicrosoft(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 1)  
        self.SetStartDate(2020, 1, 1)
        self.SetCash(10000)  #  10,000$ are available
        
        self.msft = self.AddEquity("MSFT", Resolution.Daily)  # Add MSFT to securities
        self.sell_toggle = True
        self.entry = None
        
    def OnData(self, data):
        ''' OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
            Arguments:
                data: Slice object keyed by symbol containing the stock data
        '''
      
        # Check if we're not invested and then invest everything in Microsoft.      
        if not self.Portfolio.Invested and self.sell_toggle:
            self.SetHoldings("MSFT", 1)
            self.entry = self.msft.Open
            self.StopMarketOrder("MSFT",-10,self.entry*1.5)
            self.sell_toggle = False

**Task** Buy Microsoft on 1.1.2000 and sell on 1.1.2010. Dont do anything after. <br />
Hint: Use Schedule.On(...) whith a specific date rule: (https://www.quantconnect.com/docs/algorithm-reference/scheduled-events)

In [ ]:
class BuyMicrosoftAndSell(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2000, 1, 1)  # We want to invest on the first January 2000
        self.SetCash(10000)  #  10,000$ are available
        
        # Add microsoft to securities
        self.AddEquity("MSFT", Resolution.Daily)  
        
        # Add Event to sell on 1.1.2010
        self.Schedule.On(self.DateRules.On(2010, 1, 1), self.TimeRules.At(9,30), self.Sell)
        
        self.invest = True

        
    def OnData(self, data):
        ''' OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
            Arguments:
                data: Slice object keyed by symbol containing the stock data
        '''
      
        # Check if we're not invested and then invest everything in Microsoft.
        # Only invest once thus the self.invest toggle
        if not self.Portfolio.Invested and self.invest:
            self.SetHoldings("MSFT", 1)
            
    def Sell(self):
        """
        This function is triggered on the 1.1.2010. Thus we sell Microsoft
        """
        self.Liquidate("MSFT")
        self.invest = False

**Task** Invest 100 \\$ each month into the SP500 (SPY) starting on 1.1.2000. If the stock price is larger than 100 \\$ add money to a cash reserve and invest next month with the full cash reserve

In [ ]:
class MonthlyBuy(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(10000)
        self.SetStartDate(2000,1,1) # Start on 1.1.2000
        self.spy = self.AddEquity("SPY", Resolution.Daily)
        
        # Add monthly event to buy spy shares
        self.Schedule.On(self.DateRules.MonthStart(), self.TimeRules.At(9,30), self.Buy)
        
        self.monthly_buy = 100
        self.cash_reserve = 100  # Used to store our money. Additional money is added if stock is too expensive

    def Buy(self):
        """
        Buy function which is called on the first trading day of each month
        """

        # If we run out of money, dont buy anymore! End of investing
        if self.Portfolio.Cash < self.spy.Open:
            self.Log("Not enough money! Stop investing")
            return
        
        # If the stock price is larger than our cash reserve,
        # we increase our cash reserve and dont buy anything this month
        elif self.spy.Open > self.cash_reserve:
            self.cash_reserve += self.monthly_buy

            self.Log(f"Stock is too expensive ({self.spy.Open} $), add {self.monthly_buy} $ to cash reserve. \
                         Current amount: {self.cash_reserve} $")
            return
        
        # Calculate how many shares we can buy with the amount in our cash reserve
        shares_to_buy = int(self.cash_reserve / self.spy.Open)
        self.Log(f"Buying {shares_to_buy} of {self.spy.Symbol}. Remaining money: {self.Portfolio.Cash} $")
        self.MarketOrder(self.spy.Symbol, shares_to_buy)
        # Reset cash reserve
        self.cash_reserve = self.monthly_buy

**Question** Buy Apple and Microsoft, both with a leverage of 2, on the 1.1.2010 <br />
Hint: You need to set the leverage for each share you want to invest in. Keep in mind that we want to buy both shares with a leverage of 2, thus we need at least 4 times as much cash as we defined in **SetCash**. <br />
Is a leverage of 4 enough? Try it out

In [ ]:
class Leverage(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 1)  # Invest on 1.1.2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.msft = self.AddEquity("MSFT", Resolution.Daily)
        
        # Use a 5x leverage for both stocks to make sure that both trades can be made with a 2x leverage
        self.apple.SetLeverage(5)
        self.msft.SetLeverage(5)
        
    def OnData(self, data):
        
        if  not self.Portfolio.Invested:
            self.SetHoldings(self.apple.Symbol, 2)
            self.SetHoldings(self.msft.Symbol, 2)


**Question** You decided to short amazon 5 years ago, on the 1.1.2016. Was this a good idea? <br />
Hint: Search for "amazon ticker" to find the right equity ticker name

In [ ]:
class Short(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2016, 1, 1)  # We want to short on 1.1.2016
        self.SetCash(10000)  #  10,000$ are available
        
        self.amazon = self.AddEquity("AMZN", Resolution.Daily)


    def OnData(self, data):
        
        if  not self.Portfolio.Invested:
            # Calculate how many shares we can sell
            shares_to_buy = int(self.Portfolio.Cash / self.amazon.Open)
            self.Log(f"Selling {shares_to_buy} of {self.amazon.Symbol}. Remaining money: {self.Portfolio.Cash} $")
            self.MarketOrder(self.amazon.Symbol, -shares_to_buy)
    
